<a href="https://colab.research.google.com/github/ananya0-0/Hand-Gesture-Recognition-/blob/main/Hand_Gesture_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GITHUB REPO SETUP


In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"ananya365","key":"30292cd59f8111026a3649b4006c0b95"}'}

In [ ]:
import os
!git init
!git config --global user.email "ananychauhan365@gmail.com"
!git config --global user.name "ananya365"
#to exclude large files --> .gitignore
!echo "model.p" > .gitignore
!echo "data.pickle" >> .gitignore
!echo "__pycache__/" >> .gitignore

!git add .gitignore
!git commit -m "chore: add .gitignore to exclude models and data"
print("git repository initialized and configured.")

In [ ]:
!pip install streamlit mediapipe opencv-python-headless scikit-learn
!npm install localtunnel

UPLOADING KAGGLE API TOKEN

In [ ]:
!pip install kaggle

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d grassknoted/asl-alphabet

!unzip -o -q asl-alphabet.zip -d asl_alphabet_train

print("Dataset unzipped successfully!")

In [ ]:
print("Committing .gitignore file...")
!git add .gitignore
!git commit -m "chore: Add .gitignore to exclude models and data"
print("Commit 1 complete.")

REQUIRED ML PIPELINES AND DATA TRAINING


In [6]:
%%writefile train_model.py
import cv2
import os
import numpy as np
import mediapipe as mp
import pickle
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

dataset_dir = '/content/asl_alphabet_train/asl_alphabet_train'

model_save_dir = '/content/ASL_Model'
model_path = os.path.join(model_save_dir, 'asl_classifier.pkl')
label_map_path = os.path.join(model_save_dir , 'label_map.pkl')

#feature extraction function
def extract_features(img_path, hands_model):
    img = cv2.imread(img_path)
    if img is None:
        return None

    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands_model.process(img_rgb)

    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        landmark_vector = []

        wrist_x = hand_landmarks.landmark[0].x
        wrist_y = hand_landmarks.landmark[0].y

        for landmark in hand_landmarks.landmark:
            landmark_vector.extend([landmark.x - wrist_x, landmark.y - wrist_y])
        return np.array(landmark_vector)

    return None

#training function
def train_model():
    features = []
    labels = []
    label_map = {}
    current_label_id = 0

    #initializing hands model
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.7)

    print(f"Loading data from: {dataset_dir}")

    if not os.path.exists(dataset_dir):
        print(f"Error: Dataset directory not found at {dataset_dir}")
        hands.close()
        return

    #creating label map
    all_class_dirs = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]

    for class_name in sorted(all_class_dirs):
        if class_name not in label_map:
            label_map[class_name] = current_label_id
            current_label_id += 1

    #iterating through classes and extract features
    for class_name in sorted(all_class_dirs):
        class_path = os.path.join(dataset_dir, class_name)
        label_id = label_map[class_name]
        print(f"Processing class: {class_name} (ID: {label_id})")

        image_files = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

        for image_file in image_files:
            image_path = os.path.join(class_path, image_file)
            feature_vector = extract_features(image_path, hands)

            if feature_vector is not None:
                features.append(feature_vector)
                labels.append(label_id)

    hands.close()

    print(f"\nFeature extraction complete. Total samples: {len(features)}")
    if len(features) == 0:
        print("No features extracted. Check dataset path and image content.")
        return

    #model training
    X = np.array(features)
    y = np.array(labels)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    mlp = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=42, verbose=True)
    mlp.fit(X_train, y_train)

    y_pred = mlp.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Test Accuracy: {accuracy * 100:.2f}%")

    os.makedirs(model_save_dir, exist_ok=True)
    #pickle file
    with open(model_path, 'wb') as file:
        pickle.dump(mlp, file)
    print(f"Trained Model saved to: {model_path}")

    id_to_sign = {v: k for k, v in label_map.items()}
    with open(label_map_path, 'wb') as file:
        pickle.dump(id_to_sign, file)
    print(f"Label Map saved to: {label_map_path}")

if __name__ == "__main__":
    train_model()

Overwriting train_model.py


In [7]:
!git add train_model.py
!git commit -m "adding full training pipeline"

[master 8e02e67] adding full training pipeline
 1 file changed, 6 insertions(+), 6 deletions(-)


CREATING THE STREAMLIT APP


In [8]:
%%writefile app.py
import streamlit as st
import cv2
import numpy as np
import mediapipe as mp
import pickle
from PIL import Image
import os

st.set_page_config(page_title="Hand Gesture Recognition", layout="centered")

st.title("🖐️ HAND GESTURE RECOGNITION")
st.caption("Powered by MediaPipe & Scikit-Learn")

#loading resources
@st.cache_resource
def load_resources():
    model_save_dir = '/content/ASL_Model'
    model_path = os.path.join(model_save_dir, 'asl_classifier.pkl')
    label_map_path = os.path.join(model_save_dir, 'label_map.pkl')

    #load model
    try:
        with open(model_path, 'rb') as file:
            model = pickle.load(file)
    except FileNotFoundError:
        return None, f"File '{model_path}' not found. Please ensure the model is trained and saved."
    except Exception as e:
        return None, f"Error loading model from {model_path}: {e}"

    #load label map
    try:
        with open(label_map_path, 'rb') as file:
            labels_dict = pickle.load(file)
    except FileNotFoundError:
        return None, f"File '{label_map_path}' not found. Please ensure the label map is saved."
    except Exception as e:
        return None, f"Error loading label map from {label_map_path}: {e}"

    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(
        static_image_mode=True,
        max_num_hands=1,
        min_detection_confidence=0.5)
    return (model, hands, labels_dict), None

resources, error = load_resources()

if error:
    st.error(error)
    st.stop()

model, hands, labels_dict = resources

#the logic behind prediction
def process_image(image):
    """
    Extracts landmarks and normalizes them exactly like the training script.
    """
    data_aux = []
    x_ = []
    y_ = []

    #converting pil to opencv format
    img_array = np.array(image)
    img_rgb = cv2.cvtColor(img_array, cv2.COLOR_BGR2RGB)

    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]

        #collect all X and Y coordinates to find boundaries
        for i in range(len(hand_landmarks.landmark)):
            x = hand_landmarks.landmark[i].x
            y = hand_landmarks.landmark[i].y
            x_.append(x)
            y_.append(y)

        #normalizing features (subtracting min_x & min_y)
        for i in range(len(hand_landmarks.landmark)):
            x = hand_landmarks.landmark[i].x
            y = hand_landmarks.landmark[i].y
            data_aux.append(x - min(x_))
            data_aux.append(y - min(y_))

        return data_aux, results.multi_hand_landmarks

    return None, None

#UI interface
st.write("Capture an Image")
img_file_buffer = st.camera_input("Make a gesture with your hand")

if img_file_buffer is not None:
    image = Image.open(img_file_buffer) #loading image

    features, landmarks = process_image(image) #running prediction

    if features:
        try:
            prediction = model.predict([np.asarray(features)])
            predicted_character = labels_dict.get(int(prediction[0]), str(int(prediction[0])))
           # result display
            st.success(f"Prediction: **{predicted_character}**")

            # (Optional) Visualize landmarks on the image could go here

        except ValueError as e:
            st.error(f"Shape Mismatch: The model expects different features than provided. Error: {e}")
    else:
        st.warning("No hand detected. Please ensure your hand is clearly visible.")

Overwriting app.py


In [9]:
!curl ipv4.icanhazip.com

34.12.125.198


In [10]:
#launching the app
print("Your app is running! Click the link below ending in .loca.lt")
!streamlit run app.py & npx localtunnel --port 8501

Your app is running! Click the link below ending in .loca.lt
⠙

your url is: https://nasty-moments-type.loca.lt

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.12.125.198:8501

2025-12-28 11:19:50.557713: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-28 11:19:50.565357: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-28 11:19:50.590089: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766920790.630177   18611 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766920790.643684   1